In [1]:
import numpy as np
import spacy as sp
import pandas as pd

In [ ]:
nlp = sp.load('en_core_web_sm')

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab, attr = "LOWER")

Feature extraction uses phrasematcher from the spacy module to determine what events occured. 
e.g. red odu, recent weather events. 
The phrasematcher works based on tokenisation, this allows words of strings be distinguish from each other.

E.g.

"red odu? na" vs "red odu? n"


The attr = "LOWER" allows for case insensitive matches

Using the spacy 3.0 requires python 3.8(maybe higher version work as well) 


Note: Please check file type matches

In [ ]:
def preprocessing(df):
    
    
    for i in np.arange(0, len(df.index)): # len(df.index) is the number of rows of df

        """
        Code is not that important. This was initially done to avoid tokenising the 
        format specifier which would have an effect on the phrasematcher.

        """

        df.loc[i, "description"] = str(df["description"][i]).replace("\n", " ")

        df.loc[i, "description"] = str(df["description"][i]).replace("\r", " ")


        df.loc[i, "worklog"] = str(df["worklog"][i]).replace("\n", " ")

        df.loc[i, "worklog"] = str(df["worklog"][i]).replace("\r", " ")

        df.loc[i, "worklog"] = str(df["worklog"][i]).replace("\t", " ")



        # Remove multiple spaces

        df.loc[i, "description"] = ' '.join( str(df["description"][i]).split() ) #Allows for whitespace insensitive matches

        df.loc[i, "worklog"] = ' '.join( str(df["worklog"][i]).split() ) #Allows whitespace case insensitive matches
        
        
        return (df["description"][:], df["worklog"][:]) 

In [2]:
"""
Output is binary matrix appended to the csv file for each feature in phrase_list

"""

def NLP_one_hot_encoding(df, phrase_list, file_name, label_name):
    

    df.loc[:, "description"], df.loc[:, "worklog"] = preprocessing(df)
    
    column_list = [*df.columns, *phrase_list]
    
    
    patterns = [nlp(text) for text in phrase_list]
    
    
    for j in patterns:
    

        matcher.add("pattern_name", None, *[j])

        matches_found = np.array([])

        feature_name = np.tile(0, (1, len(df.index)))   
    
    
        for i in np.arange(0, len(df.index)): # len(df.index) is the number of rows of df 

            array_size = np.size(matches_found)

            matches_found = np.append(matches_found, matcher( nlp ( str(df[label_name][i])) ) )

            if(array_size < np.size(matches_found) ):
                feature_name[0][i] = 1
            
    
        df = pd.concat([df, pd.DataFrame(np.transpose(feature_name))], axis=1)
        
        
        matcher.remove("pattern_name")
        
     
    df.columns =  column_list
    
    df.to_csv(file_name, index = False)

Apply the one-hot encoding

In [ ]:
df = pd.read_csv(file_name)
NLP_one_hot_encoding(df, ["some_string"], "some_file_name", "description")
NLP_one_hot_encoding(df, ["some_string"], "some_file_name", "worklog")